In [1]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
!python3 -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 3.7 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [3]:
from fractions import Fraction
import unicodedata

def str_to_fraction(data: str):
    sum = Fraction()
    table = str.maketrans({u'⁄': '/'})
    data = unicodedata.normalize('NFKD', data).translate(table).split()
    for val in data:
        sum += Fraction(val)
    return sum

def fraction_to_str(frac: Fraction):
    if frac.denominator == 1:
        return str(frac.numerator)
    elif frac.numerator >= frac.denominator:
        return ' '.join([str(frac.numerator // frac.denominator),
                         str(Fraction(frac.numerator % frac.denominator,
                                      frac.denominator))])
    else:
        return str(frac)

In [4]:
class Ingredient:
    '''Struct holding ingredient information'''

    def __init__(self, name: str | None = None,
                 quantity: Fraction | None = None,
                 unit: str | None = None):
        self.name = name
        self.quantity = quantity
        self.unit = unit

class Step:
    '''Struct holding step information'''

    def __init__(self, text: str):
        self.text = text
        self.ingredients = []
        self.tools = []
        self.methods = []
        self.times = []
    
class Recipe:
    '''Struct holding recipe information'''

    def __init__(self):
        self.ingredients = []
        self.steps = []
        self.other = {}

In [ ]:
import re
import spacy
nlp = spacy.load("en_core_web_md")

def parse_and_add_step(instr: str, recipe: Recipe) -> None:
    '''Parses an instruction into maybe more steps and adds to recipe.'''

    # Split recipe instruction into "sentences" for SpaCy parser
    steps = re.split(r'(?:\.|;|, then)\s+', instr)

    for text in steps:

        # Skip if empty
        if not text:
            continue

        # Form a nice sentence for the step text, and initialize step
        text = ''.join([text[0].upper(), text[1:len(text)]])
        if not text.endswith('.'):
            text = ''.join([text, '.'])
        step = Step(text)

        # Uncapitalize the first letter. This prevents SpaCy from reading the
        # first word as a proper noun (imperative sentences are less common in
        # its dataset).
        text = ''.join([text[0].lower(), text[1:len(text)]])

        # Based on the dependency + part of speech tagging, extract necessary
        # information.
        doc = nlp(text)
        for token in doc:

            # If the parser didn't interpret this as an imperative sentence,
            # correct it.
            if token.dep_ == 'ROOT' and token.head.pos_ == 'NOUN':
                doc[0].dep_ = 'ROOT'
                token.dep_ = 'dobj'
                step.methods.append(doc[0].text)

            # Perform extraction
            dep = token.dep_
            match dep:
                case 'ROOT':
                    step.methods.append(token.text)
                case 'conj':
                    if token.head.pos_ == 'VERB':
                        step.methods.append(token.text)

        # Save step to recipe
        recipe.steps.append(step)

In [6]:
from html.parser import HTMLParser
from enums import RecipeSource, HTMLTag

class RecipeHTMLParser(HTMLParser):
    '''HTML parser that handles recipes'''

    def __init__(self, source: RecipeSource, convert_charrefs: bool = True) -> None:
        # Initialize class, setting recipe to empty
        self.source = source
        self.recipe = Recipe()
        self.current_tag = HTMLTag.UNKNOWN
        super().__init__(convert_charrefs=convert_charrefs)
    
    def handle_starttag(self, tag: str, attrs: list[tuple[str, str | None]]) -> None:
        # Save the current tag
        self.current_tag = HTMLTag.from_tag(self.source, tag, attrs)
        match self.current_tag:
            case HTMLTag.INGREDIENT:
                self.ingredient = Ingredient()
        return super().handle_starttag(tag, attrs)
    
    def handle_data(self, data: str) -> None:
        # Handle text between tags as appropriate
        match self.current_tag:
            case HTMLTag.OVERVIEW_LABEL:
                self.label = data.lower().strip(':,.! \n\t')
            case HTMLTag.OVERVIEW_TEXT:
                self.recipe.other[self.label] = data.strip()
            case HTMLTag.INGREDIENT_QUANTITY:
                self.ingredient.quantity = str_to_fraction(data.strip())
            case HTMLTag.INGREDIENT_UNIT:
                self.ingredient.unit = data.strip()
            case HTMLTag.INGREDIENT_NAME:
                self.ingredient.name = data.strip()
                self.recipe.ingredients.append(self.ingredient)
            case HTMLTag.STEP:
                parse_and_add_step(data.strip(), self.recipe)
        return super().handle_data(data)

    def handle_endtag(self, tag: str) -> None:
        # Reset tag
        self.current_tag = HTMLTag.UNKNOWN
        return super().handle_endtag(tag)

In [7]:
import re
import requests

def get_recipe_from_url(url: str) -> Recipe | None:
    '''Retrieves the text of a recipe from a given URL'''

    # Find recipe source; return None if unsupported
    source = RecipeSource.from_url(url)
    if source == RecipeSource.UNKNOWN:
        return None

    # Add appropriate HTTPS tag if not there
    if not re.match(r'https://www\.', url):
        if re.match(r'www\.', url):
            url = ''.join(['https://', url])
        else:
            url = ''.join(['https://www.', url])
    
    # Get the recipe from the page
    with requests.get(url) as f:
        parser = RecipeHTMLParser(source)
        parser.feed(f.text)
        return parser.recipe

In [12]:
recipe = get_recipe_from_url("https://www.allrecipes.com/recipe/19644/moussaka/")
for step in recipe.steps:
    print(step.text, step.methods)

Lay eggplant slices on paper towels. ['lay']
Sprinkle lightly with salt. ['sprinkle']
Let sit for 30 minutes to draw out moisture. ['sit']
Pat dry with paper towels. ['pat']
Warm olive oil in a skillet over high heat. ['warm']
Fry eggplant until browned, 2 to 3 minutes per side. ['fry']
Drain on paper towels. ['drain']
Set aside. ['set']
Melt 1 tablespoon butter in a large skillet over medium heat. ['melt']
Stir in ground beef, onions, and garlic. ['stir']
Season with salt and black pepper. ['season']
Cook and stir until beef is browned, 8 to 10 minutes. ['cook', 'stir']
Add parsley, fines herbs, cinnamon, and 1/4 teaspoon nutmeg. ['add']
Pour in tomato sauce and wine. ['pour']
Mix well. ['mix']
Simmer for 20 minutes. ['simmer']
Allow to cool. ['allow']
Stir in beaten egg. ['stir']
Scald milk in a saucepan over medium heat. ['scald']
At the same time, melt 1/2 cup butter in a large skillet over medium heat. ['melt']
Whisk flour into butter until smooth. ['whisk']
Lower heat. ['lower']


In [10]:
import spacy
nlp = spacy.load("en_core_web_md")

# text = "lay eggplant slices on paper towels"
# text = "sprinkle lightly with salt."
# text = "cook and stir until beef is browned, 8 to 10 minutes"
# text = "stir in ground beef, onions, and garlic"
# text = "meanwhile, place ground beef, garlic, oregano, garlic powder, salt, and black pepper in a large skillet over medium heat"
# text = "add lasagna noodles and cook for 10 minutes or until al dente"
# text = "preheat the oven to 350 degrees F (175 degrees C)"
text = "season béchamel sauce with salt and white pepper"

doc = nlp(text)
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])
print()
for chunk in doc.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
            chunk.root.head.text)

season compound sauce NOUN []
béchamel compound sauce NOUN []
sauce ROOT sauce NOUN [season, béchamel, with]
with prep sauce NOUN [pepper]
salt nmod pepper NOUN [and, white]
and cc salt NOUN []
white conj salt NOUN []
pepper pobj with ADP [salt]

season béchamel sauce sauce ROOT sauce
salt and white pepper pepper pobj with
